# 1) Creation de merge raw metrics dataset 

Ce notebook génère :

- 1 fichier csv "merge_raw_metrics_dataset.csv" qui fusionne les colonnes fractionnées avec le dataset d'origine
- 1 fichier json "metrics_events_dict.json" pour lister les code d'identification des évènements

## Imports

In [ ]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
from azure_blob import download_blob_file

## 1. Création dataset metrics

In [ ]:
# source path to raw metrics dataset
filename = 'metrics.csv'
path = '../data/'
source_csv = os.path.join(path, filename)
# target path to save metrics dictionnaire
save_json ='../data/metrics/metrics_events_dict.json'
# target path to save merge raw metrics dataset
save_csv = '../data/metrics/merge_raw_metrics_dataset.csv'

### a) Import des données brutes

In [ ]:
# téléchargement dans le repertoire 'data' d'un fichiers 'csv' depuis le blob Azure
download_blob_file(file_name=filename, local_path=path)

In [ ]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(filepath_or_buffer=Path(source_csv)).sort_values(by='created_at')
metrics_df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
metrics_df.info()

In [ ]:
# suppression des colonnes ne contenant aucune valeurs
metrics_df = metrics_df.dropna(axis=1)
# suppression de la colonne machineId
metrics_df = metrics_df.drop('machineId', axis=1)
# visualisation des 3 premières lignes
metrics_df.head(3)

### b) Fractionnement des colonnes contenant des listes

In [ ]:
# on verifie le type des valeurs contenu dans les colonnes de type objet contenant des listes
print('modules :', type(metrics_df.modules.loc[0]))
print('events :', type(metrics_df.events.loc[0]))
print('connected_operators :', type(metrics_df.connected_operators.loc[0]))

In [ ]:
# fonction retournant le dataframe d'une colonne fractionnée
# col=colonne à fractionner
# df=dataframe source
# data=dict des colonnes du df à conserver dans le df à retourner
def convert_col_to_df(col, df, data=None):
    
    # création du dictionnaire de données vide
    if data == None :
        data = {}
    # ou liste des clés du dictionnaire input
    else :
        data_keys = list(data.keys())

    # on converti le type des valeurs str en list
    if not isinstance(df[col].loc[0], list):
        df[col] = df[col].apply(lambda x : json.loads(x))

    # liste des clés du dictionnaire de la colonne à partir de la première occurence
    # on recherche la première occurence non vide et de type list 
    # pour l'affecter à une variable first
    for i in range(0, (len(df[col]))):
        value = df[col].loc[i]
        if len(value) > 0 and isinstance(value, list):
            first = value[0]
            print('first : ', type(first), first)
            break

    # on liste les clés du dictionnaire de l'occurence
    col_keys = first.keys()
    for ck in col_keys :
        data[ck] = []

    # on itére dans la serie pour récupérer les valeurs et les stocker dans le dictionnaire data
    for i in range(df.index.start, df.index.stop):
        # evaluation des valeurs 'str' en 'list'
        values = df[col].loc[i]
        if isinstance(values, list) and len(values) > 0 :
            # ajout des valeurs dans le dictionnaire 'd'
            for value in values :
                for k in value.keys():
                    data[k].append(value.get(k))
                for dk in data_keys:
                    data[dk].append(df[dk].loc[i])

    # re-assignation de la variable df
    df = pd.DataFrame(data)

    return df

In [ ]:
# id temoin pour vérifier le fractionnement et la fusion des colonnes
check_id = 4170152

In [ ]:
# détail de la ligne témoin
check_line = metrics_df[metrics_df.id == check_id]
print('######## id %d ######## ' %check_id)
print(check_line.values)
print('######## id %d modules details ######## ' %check_id)
print(json.loads(check_line.modules.values[0])[0].get('counters'))
print(json.loads(check_line.modules.values[0])[1].get('counters'))
print('######## id %d events details ######## ' %check_id)
print(json.loads(check_line.events.values[0])[0])
print(json.loads(check_line.events.values[0])[1])

#### 1) Colonne 'connected_operators'

In [ ]:
# creation d'un dataframe connected_operators (~42s)
connected_operators_df = convert_col_to_df('connected_operators', metrics_df, {'id':[]})
print(connected_operators_df.info())
connected_operators_df.head(2)

#### 2) Colonne 'events'

In [ ]:
# creation d'un dataframe events (~20s)
events_df = convert_col_to_df('events', metrics_df, {'id':[]})
print(events_df.info())
events_df.head(2)

##### Identification

In [ ]:
# liste des codes d'identification
identification_codes_list = events_df['identification'].unique()
np.sort(identification_codes_list)

In [ ]:
# liste des évènements uniques
identification_dict = {}
c = 1
id_list = []
for i in range(events_df.index.start, events_df.index.stop):
    id = events_df.identification.loc[i]
    if id not in id_list:
        id_list.append(id)
        identification_dict[id] = events_df.message.loc[i]
        c += 1
identification_dict

##### Source

In [ ]:
# liste des sources
source_list = events_df['source'].unique()
np.sort(source_list)

##### Criticality

In [ ]:
# liste des sources
criticality_list = events_df['criticality'].unique()
np.sort(criticality_list)

##### Events Json dict

In [ ]:
# on sauvegarde la liste des codes d'identification d'event
with open(file=Path(save_json), mode="r+", encoding='utf-8') as jsonFile:
    data = json.load(jsonFile)
    data['identification'] = identification_dict
    data['criticality'] = list(np.sort(criticality_list)),
    data['source'] = list(np.sort(source_list))
    jsonFile.seek(0)
    json.dump(data, jsonFile, indent=4, ensure_ascii=False)
    jsonFile.close()

#### 3) Colonne 'modules'

In [ ]:
# creation d'un dataframe modules (~1m15s)
modules_df = convert_col_to_df('modules', metrics_df, {'id':[]})
print(modules_df.info())
modules_df.head(2)

##### Colonne counters

In [ ]:
# creation d'un dataframe counters (~2m7s)
counters_df = convert_col_to_df('counters', modules_df, {'type':[], 'id': []})
print(counters_df.info())
counters_df.head(2)

In [ ]:
counters_check_line = counters_df[counters_df.id == check_id]
counters_check_line

### c) Fusion des dataframes des colonnes fractionnées

#### 1) Merge modules et counters

In [ ]:
merge_modules_df = pd.merge(modules_df, counters_df, on=['id','type'], suffixes=['','_counter'])
merge_modules_df = merge_modules_df.drop(['counters'], axis=1)
module_check_line = merge_modules_df[merge_modules_df.id == check_id]
module_check_line

In [ ]:
merge_modules_df.info()

#### 2) Merge operators

In [ ]:
merge_operators_df = pd.merge(merge_modules_df, connected_operators_df, on='id', suffixes=['','_op'])
op_check_line = merge_operators_df[merge_operators_df.id == check_id]
op_check_line

#### 3) Merge events

In [ ]:
merge_events_df = pd.merge(merge_operators_df, events_df, how='outer', on='id', suffixes=['','_event'])
events_check_line = merge_events_df[merge_events_df.id == check_id]
events_check_line

In [ ]:
merge_events_df.info()

#### 4) Merge metrics

In [ ]:
merge_metrics_df = pd.merge(merge_events_df, metrics_df, how='outer', on='id', suffixes=['','_metrics'])
merge_metrics_df = merge_metrics_df.drop(['status','connected_operators','modules','events'], axis=1)
metrics_check_line = merge_metrics_df[merge_metrics_df.id == check_id]
metrics_check_line

In [ ]:
merge_metrics_df.info()

In [ ]:
merge_metrics_df.to_csv(path_or_buf=Path(save_csv))